In [1]:
import pandas as pd
import numpy as np
import os
import ast

# Datos

In [2]:
#Subir un directorio
os.chdir("..")

In [3]:
places_details = pd.read_csv('data/places_details_table.csv')
places_reviews = pd.read_csv('data/reviews_google_final.csv')

In [22]:
external_types = pd.read_csv('data/places_types.csv')

# Formatear tablas

## Places reviews

In [4]:
places_reviews_final = places_reviews.copy()
places_reviews_final = places_reviews_final[['place_id', 'url','datetime', 'rating', 'language', 'text', 'reviewer'
                                            ]]

In [5]:
#Parsear columna de fecha
places_reviews_final['timestamp'] = pd.to_datetime(places_reviews_final['datetime'])

In [6]:
places_reviews_final['date'] = places_reviews_final['timestamp'].dt.date

In [7]:
places_reviews_final['time'] = places_reviews_final['timestamp'].dt.time

In [8]:
#Tirar columnas no necesarias
places_reviews_final = places_reviews_final.drop(columns=['datetime'])

In [9]:
#Guardar
places_reviews_final.to_csv('data/places_reviews_processed.csv', index=False)

## Obtener estadísticas de reviews

In [10]:
places_reviews_final_grouped = places_reviews_final.groupby(['place_id',
                                                             'rating']).agg({'url':'count'}).reset_index()

In [11]:
places_reviews_final_pivot = pd.pivot_table(places_reviews_final_grouped, values='url', index=['place_id'],
                                        columns=['rating'], aggfunc=np.sum).reset_index()

In [12]:
#Substituir nulos
places_reviews_final_pivot = places_reviews_final_pivot.fillna(0)

In [13]:
places_reviews_final_pivot['total_reviews'] = places_reviews_final_pivot['1']+places_reviews_final_pivot['2']+places_reviews_final_pivot['3']+places_reviews_final_pivot['4']+places_reviews_final_pivot['5']

In [14]:
places_reviews_final_pivot['5_percent_total'] = places_reviews_final_pivot['5']/places_reviews_final_pivot['total_reviews']

In [15]:
top_rated = places_reviews_final_pivot[['place_id', '5_percent_total']]

## Places details

In [16]:
places_details_final = places_details.copy()
places_details_final = places_details_final.rename(columns={'formatted_address.1':'contact_number'})

In [17]:
places_details_final = places_details_final[['place_id', 'name','business_status', 'latitude', 'longitude',
                                             'user_ratings_total', 'rating', 'website', 'price_level',
                                             'photos', 'sunday_open', 'sunday_close', 'monday_open', 
                                             'monday_close', 'tuesday_open', 'tuesday_close',
                                             'wednesday_open', 'wednesday_close','thursday_open', 
                                             'thursday_close', 'friday_open', 'friday_close',
                                             'saturday_open', 'saturday_close'
                                            ]]

In [18]:
places_details_final = places_details_final.fillna('')

# Types

In [19]:
types = places_details.copy()
types = types[['place_id', 'types']]

In [20]:
types_df = pd.DataFrame()

for i in range(0, len(types)):
    x = ast.literal_eval(types['types'][i])
    place_id = types['place_id'][i]
    temp_list = [place_id]
    try:
        temp_list.append(x[0])
    except:
        temp_list.append('')
    try:
        temp_list.append(x[1])
    except:
        temp_list.append('')
    try:
        temp_list.append(x[2])
    except:
        temp_list.append('')
    try:
        temp_list.append(x[3])
    except:
        temp_list.append('')
    
    temp_df = pd.DataFrame(columns=['place_id', 'type_1', 'type_2', 'type_3', 'type_4'])
    a_series = pd.Series(temp_list, index = temp_df.columns)
    types_df = types_df.append(a_series, ignore_index=True)


## Unir tablas

In [24]:
# Top rated
places_details_final_2 = pd.merge(places_details_final, top_rated, on='place_id', how='left')

In [25]:
#Types
places_details_final_2 = pd.merge(places_details_final, types_df, on='place_id', how='left')

In [26]:
#External types
places_details_final_2 = pd.merge(places_details_final, external_types, on='place_id', how='left')

In [29]:
# Tirar duplicados 
places_details_final_2 = places_details_final_2.drop_duplicates(subset=['place_id'])
#Tirar nas
places_details_final_2 = places_details_final_2.dropna(subset=['tipo_lugar'])

In [31]:
places_details_final_2.to_csv('data/places_details_processed.csv', index=False)